In [1]:
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix, vstack
from tqdm.notebook import tqdm

In [ ]:
# LDPC decoder for the BEC, inner loop
# completely vectorized but still relatively slow
# based on a sparse matrix as input
def decode_LDPC(L, H, iterations):
    m, n = H.shape
    H = csr_matrix(H)
    row_i, col_i = H.nonzero() # get row and column indices

    # initialize variable to check node messages with channel output
    VtoC = L[col_i].copy()

    # numeric stability
    epsilon = 1e-12

    # main iterations
    for _ in range(iterations):
        # compute check to variable sum(CtoV,1)node messages
        VtoC_sign = np.where(VtoC < 0, -1, 1)
        VtoC_abs = np.abs(VtoC)

        phiVtoC = np.log(1 / np.tanh(VtoC_abs / 2) + epsilon)
        phiVtoC_sum = np.bincount(row_i, weights=phiVtoC, minlength=m)

        # multiply signs
        totalsign_VtoC = np.ones(m)
        np.multiply.at(totalsign_VtoC, row_i, VtoC_sign)
        
        CtoV_abs = np.log(1 / np.tanh(((phiVtoC_sum[row_i] - phiVtoC) / 2)) + epsilon)

        CtoV_sign = totalsign_VtoC[row_i] * VtoC_sign
        CtoV = CtoV_sign * CtoV_abs
        
        # compute variable to check node messages, pretty simple
        CtoV_sum = np.bincount(col_i, weights=CtoV, minlength=n)
        VtoC = L[col_i] + CtoV_sum[col_i] - CtoV

        # stopping criterion, all parity checks are fulfilled
        L_total = CtoV_sum + L
        
        # binary decision
        xh = (L_total < 0).astype(int)
        if np.all((H @ xh) % 2 == 0):
            # all parity-checks fulfilled?
            break
        
    return xh

# LDPC decoder using the full (simplified) update rule, inner loop
# completely vectorized but still relatively slow
# based on a non-sparse parity-check matrix as input
def decode_LDPC_nosparse(L, H, iterations):
    m, n = H.shape

    # initialize variable to check node messages with channel output
    VtoC = np.tile(L.reshape(1, -1), (m, 1)) * H

    # main iterations
    for _ in range(iterations):
        # compute check to variable sum(CtoV,1)node messages
        VtoC_sign = mysign(VtoC)
        VtoC_abs = np.abs(VtoC)

        phiVtoC = phifun(VtoC_abs/2 + (1-H) * 9e9)  # mask out zero entries
        phiVtoC_sum = np.sum(phiVtoC, axis=1)

        # multiply signs
        totalsign_VtoC = np.prod(VtoC_sign, axis=1, keepdims=True)
       
        CtoV_abs = phifun((np.tile(phiVtoC_sum.reshape(-1, 1), (1, n)) * H - phiVtoC)/2 + (1-H) * 9e9)
        CtoV_sign = np.tile(totalsign_VtoC.reshape(-1, 1), (1, n)) * VtoC_sign
        CtoV = CtoV_sign * CtoV_abs       
       
        
        # compute variable to check node messages, pretty simple
        CtoV_sum = np.sum(CtoV,axis=0)
        VtoC = (np.tile(CtoV_sum + L, (m,1)) - CtoV) * H     

        # stopping criterion, all parity checks are fulfilled
        L_total = CtoV_sum + L
        
        # binary decision
        xh = (L_total < 0).astype(int)
        if np.all((H @ xh) % 2 == 0):
            # all parity-checks fulfilled?
            break
    
    return xh


# generate a parity-check matrix according to Gallager's method
# do not care about 4-cycles
def generate_Gallager(dv, dc, n):
    if n % dc != 0:
        assert False, "n must be a multiple of check node degree dc"

    rows = n // dc
    # column indices
    jj = np.arange(n)
    ii = np.repeat(np.arange(rows), dc)
    Ho = coo_matrix((np.ones_like(jj), (ii, jj)), shape=(rows,n)).tocsr()
    H = Ho.copy()
    for _ in range(dv-1):
        H = vstack([H, Ho[:, np.random.permutation(n)]])
    
    return H

def phifun(x):
    y = 9e9*np.ones_like(x)
    y[x>1e-300] = -np.log(np.tanh(x[x>1e-300]))
    return y

def mysign(x):
    y = np.ones_like(x)
    y[x<0] = -1
    return y

In [113]:
# simulate using MATLABs integrated LDPC decoder
def simulate_LDPC():
    # parameters of regular LDPC code
    dv = 3
    dc = 6

    # specify Es/N0 at which simulation takes place
    esno_dB = 1

    # number of frames to simulate
    frames = 1000

    # decodsing iterations
    iterations = 5

    # compute noise standard deviation
    sigma = np.sqrt(0.5 * 10**(-esno_dB/10))

    # channel parameter for LLR calculation
    Lc = 4*(10**(esno_dB/10))

    # generate parity-check matrix of regular LDPC code
    H = generate_Gallager(dv, dc, 1200)

    n = H.shape[1]

    # simulate all-zero codeword
    bits = np.zeros(n) 
    x = 1 - 2*bits #BPSK 0->1 | 1->-1

    errors = 0
    for _ in tqdm(range(frames)):
        y = x + sigma*np.random.randn(n)

        # calculate LLRs
        L = Lc * y

        xh = decode_LDPC(L.flatten(), H, iterations)

        # alternative using the full matrix (faster for very small matrices,
        # slower for large matrices)
        #xh = decode_LDPC_nosparse(L.flatten(), H.toarray(), iterations) 
        
        errors = errors + np.sum(xh != bits)

    BER = errors / (frames * n)
    print(f"Es/N0 = {esno_dB:.2f}: BER = {BER:.4g}\n")

In [114]:
simulate_LDPC()

  0%|          | 0/1000 [00:00<?, ?it/s]

Es/N0 = 1.00: BER = 9.167e-06

